Connectome Workbench is required: https://www.humanconnectome.org/software/connectome-workbench

In [13]:
import numpy as np
import time
import glob

birth_age_indices = np.loadtxt('indices/birth_age_full.txt', dtype=str)
scan_age_indices = np.loadtxt('indices/scan_age_full.txt', dtype=str)
all_indices = list(set(birth_age_indices.tolist() + scan_age_indices.tolist()))

all_indices.sort()

print(all_indices)


['sub-CC00051XX02_ses-7702', 'sub-CC00052XX03_ses-8300', 'sub-CC00053XX04_ses-8607', 'sub-CC00054XX05_ses-8800', 'sub-CC00055XX06_ses-9300', 'sub-CC00056XX07_ses-10700', 'sub-CC00057XX08_ses-11002', 'sub-CC00060XX03_ses-12501', 'sub-CC00061XX04_ses-13300', 'sub-CC00062XX05_ses-13801', 'sub-CC00064XX07_ses-18303', 'sub-CC00065XX08_ses-18600', 'sub-CC00066XX09_ses-19200', 'sub-CC00067XX10_ses-20200', 'sub-CC00068XX11_ses-20701', 'sub-CC00069XX12_ses-26300', 'sub-CC00071XX06_ses-27000', 'sub-CC00073XX08_ses-27800', 'sub-CC00074XX09_ses-28000', 'sub-CC00075XX10_ses-28400', 'sub-CC00078XX13_ses-29900', 'sub-CC00079XX14_ses-30000', 'sub-CC00080XX07_ses-30300', 'sub-CC00082XX09_ses-30700', 'sub-CC00083XX10_ses-30900', 'sub-CC00084XX11_ses-31201', 'sub-CC00085XX12_ses-31400', 'sub-CC00086XX13_ses-31500', 'sub-CC00087AN14_ses-31800', 'sub-CC00087BN14_ses-32100', 'sub-CC00088XX15_ses-31801', 'sub-CC00089XX16_ses-32101', 'sub-CC00091XX10_ses-32300', 'sub-CC00094AN13_ses-33500', 'sub-CC00094BN13_s

In [22]:
import os

root = r"E:/"

output_path = r"""..\data""" + '\\'
dir = "../"
continued = "data"
continued2 = 'features'
output_path = os.path.join(dir)
print("Test: ", output_path)
num_of_indices = len(all_indices)
print("Total num of indices: ", num_of_indices)
#output_path = '../data/features/'

template = True

#get preprocessing time
start_time = time.time()
time_array = []
file_size_array = []

#if sampling for training, for space reasons it's wise to limit the number of indices processed here
num_of_files = 0
for index in all_indices[1:100]:
    time_array.append(time.time())
    sub, ses = index.split('_')
    folder_location = '../rel3_dhcp_anat_pipeline/'
    test_location = os.path.join("..", "rel3_dhcp_anat_pipeline")
    print(test_location)
    print(folder_location)
    folder_list = [x[0] for x in os.walk(folder_location)]
    print("Folder location sanity check: ", folder_list[0], len(folder_list)) #this passes.
    input_path = os.path.join(folder_location, sub, ses)
    print("Input path: ", input_path)
    
    anat_path = input_path + '/anat/'
    xfm_path = input_path + '/xfm/'


    num_of_files += 1
    for hemi in ('left', 'right'):
        myelinmap = anat_path + index + '_hemi-' + hemi + '_myelinmap.shape.gii'
        curv = anat_path + index + '_hemi-' + hemi + '_curv.shape.gii'
        corr_thickness = anat_path + index + '_hemi-' + hemi + '_desc-corr_thickness.shape.gii'
        sulc = anat_path + index + '_hemi-' + hemi + '_sulc.shape.gii'
        metrics = output_path + index + '_' + hemi + '.shape.gii'
        file_string = index + "_" + hemi + ".shape.gii"

        #print("File names: " )
        #print("Myelinmap: ", myelinmap)
        #print("Curv: ", curv)
        #print("Corr: ", corr_thickness)
        #print("Sulc: ", sulc)
        print("Metrics: ", os.path.join(metrics))


        if not os.path.isfile(myelinmap):
            print(myelinmap) # not all T1w images are present in the dataset
        else:
            ico6 = 'templates/ico-6.surf.gii'
            # merge all 4 metrics
            !workbench\bin_windows64\wb_command.exe -metric-merge $metrics -metric $myelinmap -metric $curv -metric $corr_thickness -metric $sulc
            if template:
                xfm_sphere = xfm_path + index + '_hemi-' + hemi + '_from-native_to-dhcpSym40_dens-32k_mode-sphere.surf.gii'
                template_sphere = 'templates/week-40_hemi-' + hemi + '_space-dhcpSym_dens-32k_sphere.surf.gii'
                native_midthickness = anat_path + index + '_hemi-' + hemi + '_midthickness.surf.gii'
                template_midthickness = 'templates/week-40_hemi-' + hemi + '_space-dhcpSym_dens-32k_midthickness.surf.gii'
                # resample metrics from native space to template sphere (dhcpSym40) using xfm_sphere transform
                
                !workbench\bin_windows64\wb_command.exe -metric-resample $metrics $xfm_sphere $template_sphere ADAP_BARY_AREA $metrics -area-surfs $native_midthickness $template_midthickness
                # resample metrics from template sphere (dhcpSym40) to the 6th order icosphere
                !workbench\bin_windows64\wb_command.exe -metric-resample $metrics $template_sphere $ico6 BARYCENTRIC $metrics
            else:
                native_sphere = anat_path + index + '_hemi-' + hemi + '_sphere.surf.gii'
                # resample metrics from native sphere to the 6th order icosphere
                !workbench\bin_windows64\wb_command.exe -metric-resample $metrics $native_sphere $ico6 BARYCENTRIC $metrics
        #time.sleep(1)
        #filesize = os.path.getsize(metrics)
        #file_size_array.append(filesize)
    print(index, 'is ready')



file_list = glob.glob('../*.gii')
file_size_array = [os.path.getsize(file_location) for file_location in file_list]


time_changes = []
for i in range(len(time_array) - 1):
    this_time = time_array[i]
    next_time = time_array[i + 1]

    diff = next_time - this_time
    time_changes.append(diff)

total_preprocess_time = time_array[-1] - start_time
total_file_sizes = sum(file_size_array)
percentage_of_size = num_of_files / len(all_indices)
projected_file_size = total_file_sizes / percentage_of_size


average_time_per_file = sum(time_changes) / len(time_changes)
time.sleep(5)
print()
print("Total preprocess time: ", (time_array[-1] - start_time))
print("Average time per file: ", average_time_per_file)
percentage_run = num_of_files / num_of_indices
projected_time = total_preprocess_time / percentage_run
print("Projected 100% preprocess time: ", projected_time)
print()
print()
print("Len of file list: ", len(file_list))
print("Total file size in sample (mb): ", str(round(sum(file_size_array)/1048576.0, 2)))
print("Average file size (mb): ", str(round((sum(file_size_array) / len(file_size_array))/1048576.0, 2)))
print("Projected total file size (mb): ", str(round(projected_file_size/1048576.0, 2)))

#print("Average file size: ", total_file_sizes / len(file_size_array))
#print("File size written: ", total_file_sizes)
#print("Predicted full dataset size: ", projected_file_size)



Test:  ../
Total num of indices:  625
..\rel3_dhcp_anat_pipeline
../rel3_dhcp_anat_pipeline/
Folder location sanity check:  ../rel3_dhcp_anat_pipeline/ 3432
Input path:  ../rel3_dhcp_anat_pipeline/sub-CC00052XX03\ses-8300
Metrics:  ../sub-CC00052XX03_ses-8300_left.shape.gii
Metrics:  ../sub-CC00052XX03_ses-8300_right.shape.gii
sub-CC00052XX03_ses-8300 is ready
..\rel3_dhcp_anat_pipeline
../rel3_dhcp_anat_pipeline/
Folder location sanity check:  ../rel3_dhcp_anat_pipeline/ 3432
Input path:  ../rel3_dhcp_anat_pipeline/sub-CC00053XX04\ses-8607
Metrics:  ../sub-CC00053XX04_ses-8607_left.shape.gii
Metrics:  ../sub-CC00053XX04_ses-8607_right.shape.gii
sub-CC00053XX04_ses-8607 is ready
..\rel3_dhcp_anat_pipeline
../rel3_dhcp_anat_pipeline/
Folder location sanity check:  ../rel3_dhcp_anat_pipeline/ 3432
Input path:  ../rel3_dhcp_anat_pipeline/sub-CC00054XX05\ses-8800
Metrics:  ../sub-CC00054XX05_ses-8800_left.shape.gii
Metrics:  ../sub-CC00054XX05_ses-8800_right.shape.gii
sub-CC00054XX05_ses-8